# go-并发

有人把Go比作21世纪的C语言
* 第一是因为Go语言设计简单，
* 第二，21世纪最重要的就是并行程序设计，而Go从语言层面就支持了并行。

### goroutine

* goroutine是Go并行设计的核心。
* goroutine其实就是协程，但比线程更轻量，十几个goroutine可能体现在底层就是五六个线程，执行goroutine只需极少的栈内存(大概是4~5KB)，当然会根据相应的数据伸缩。
* Go语言内部帮你实现了这些goroutine之间的内存共享。

也正因为如此，可同时运行成千上万个并发任务。goroutine比thread更易用、更高效、更轻便。



### 创建goroutine

goroutine是通过Go的runtime管理的一个线程管理器。通过go关键字将一个普通的函数定义成协程并启动。如: `go hello(a, b, c)`

```
package main

import (
	"fmt"
	"runtime"
)

func say(s string) {
	for i := 0; i < 5; i++ {
		runtime.Gosched()
		fmt.Println(s)
	}
}

func main() {
	go say("world") //开一个新的Goroutines执行
	say("hello") //当前Goroutines执行
}

// 以上程序执行后将输出：
// hello
// world
// hello
// world
// hello
// world
// hello
// world
// hello
```

go关键字很方便的就实现了并发编程。 上面多个goroutine运行在同一个进程里，共享内存数据，

不过设计上我们要遵循：不要通过共享来通信，而要**通过通信来共享**。

`runtime.Gosched()`表示让CPU把时间片让给别人,下次某个时候继续恢复执行该goroutine。

默认情况下，在Go 1.5将标识并发系统线程个数的`runtime.GOMAXPROCS`的初始值由1改为了运行环境的CPU核数。

但在Go 1.5以前调度器仅使用单线程，也就是说只实现了并发。想要发挥多核处理器的并行，需要在我们的程序中显式调用 runtime.GOMAXPROCS(n) 告诉调度器同时使用多个线程。`GOMAXPROCS` 设置了同时运行逻辑代码的系统线程的最大数量，并返回之前的设置。如果`n < 1`，不会改变当前设置。

### channels-通道

goroutine运行在相同的地址空间，因此访问共享内存必须做好同步。

那么goroutine之间如何进行数据的通信呢 ? 

Go提供了一个很好的通信机制channel。channel可以与Unix shell 中的双向管道做类比：
* 可以通过它发送或者接收值。这些值只能是特定的类型：channel类型。
* 定义一个channel时，也需要定义发送到channel的值的类型。注意，必须使用make 创建channel.

```
ci := make(chan int)
cs := make(chan string)
cf := make(chan interface{})
```

channel通过操作符`<-`来接收和发送数据:
```
ch <- v    // 发送v到channel ch.
v := <-ch  // 从ch中接收数据，并赋值给v
```

### Range和close